# Señales de tráfico en Bélgica

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [4]:
train_path = "../datasets/belgian/Training"

data = pd.DataFrame()

for i in range(62):
    if i < 10:
        dir_path = "0000" + str(i)
        filepath = "GT-0000" + str(i)
    else:
        dir_path = "000" + str(i)
        filepath = "GT-000" + str(i)
    
    df = pd.read_csv(os.path.join(train_path, dir_path, filepath + ".csv"), sep=";")
    data = pd.concat([data, df], axis=0)

data.head()

,Filename,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId
0,01153_00000.ppm,141,142,12,12,128,130,0
1,01153_00001.ppm,120,123,10,10,109,113,0
2,01153_00002.ppm,105,107,9,9,96,98,0
3,01160_00000.ppm,94,105,8,9,86,95,0
4,01160_00001.ppm,128,139,11,12,117,127,0


In [5]:
data.shape

(4591, 8)

In [6]:
data["ClassId"].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61])